In [ ]:
!pip install easyocr

In [24]:
import easyocr
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import difflib

In [47]:
df = pd.read_csv('/content/Contact Information (Responses) - db.csv', low_memory=False)
df.head()

,Fecha,Nombre completo,Marca,Placa,Fotografia 1,Initial KM,Fotografia 2,Final KM,Comentario,km check,Clasificación,Kilometraje,Gasto,Budget,Fotografia 1 Path,Fotografia 2 Path,Tarifa por KM,1.19
0,11/8/2023,Jen,DODGE,JPN1,https://drive.google.com/open?id=1aajqJiEZh6N7...,"1,234",https://drive.google.com/open?id=1gtF4sguLQeN0...,"12,345",personal,NaN,Home,"11,111",13222.09,NaN,/content/drive/MyDrive/Contact Information (Fi...,/content/drive/MyDrive/Contact Information (Fi...,NaN,NaN
1,11/8/2023,Grant,KIA,PNJ2,https://drive.google.com/open?id=1GgqvsXuUFcNw...,241,https://drive.google.com/open?id=1s2j8tp5IDoqk...,245,trabajo,NaN,Work,4,4.76,NaN,/content/drive/MyDrive/Contact Information (Fi...,/content/drive/MyDrive/Contact Information (Fi...,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df[['model_km', 'Accuracy']] = 0

In [49]:
df = df.iloc[:2,:]

In [50]:
df

,Fecha,Nombre completo,Marca,Placa,Fotografia 1,Initial KM,Fotografia 2,Final KM,Comentario,km check,Clasificación,Kilometraje,Gasto,Budget,Fotografia 1 Path,Fotografia 2 Path,Tarifa por KM,1.19,model_km,Accuracy
0,11/8/2023,Jen,DODGE,JPN1,https://drive.google.com/open?id=1aajqJiEZh6N7...,"1,234",https://drive.google.com/open?id=1gtF4sguLQeN0...,"12,345",personal,NaN,Home,"11,111",13222.09,NaN,/content/drive/MyDrive/Contact Information (Fi...,/content/drive/MyDrive/Contact Information (Fi...,NaN,NaN,0,0
1,11/8/2023,Grant,KIA,PNJ2,https://drive.google.com/open?id=1GgqvsXuUFcNw...,241,https://drive.google.com/open?id=1s2j8tp5IDoqk...,245,trabajo,NaN,Work,4,4.76,NaN,/content/drive/MyDrive/Contact Information (Fi...,/content/drive/MyDrive/Contact Information (Fi...,NaN,NaN,0,0


In [25]:
# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [51]:
# Read text from an image file
def easy_rec(i):
  text = reader.readtext(f"{i}")

  # Extract the recognized text from the output
  recognized_text = []
  accuracy_image = []
  for item in text:
      recognized_text.append(item[1])
      accuracy_image.append(item[2])

  max_value = max(recognized_text)
  max_index = recognized_text.index(max_value)

  accuracy = accuracy_image[max_index]

  # Print the recognized text
  return max_value, accuracy

In [52]:
# Loop through each row in the "Fotografia 1" column and update "Max Value" and "Accuracy" columns
for index, row in df.iterrows():
    max_value, accuracy = easy_rec(row["Fotografia 1 Path"])
    df.at[index, "model_km"] = max_value
    df.at[index, "Accuracy"] = accuracy

In [53]:
df

,Fecha,Nombre completo,Marca,Placa,Fotografia 1,Initial KM,Fotografia 2,Final KM,Comentario,km check,Clasificación,Kilometraje,Gasto,Budget,Fotografia 1 Path,Fotografia 2 Path,Tarifa por KM,1.19,model_km,Accuracy
0,11/8/2023,Jen,DODGE,JPN1,https://drive.google.com/open?id=1aajqJiEZh6N7...,"1,234",https://drive.google.com/open?id=1gtF4sguLQeN0...,"12,345",personal,NaN,Home,"11,111",13222.09,NaN,/content/drive/MyDrive/Contact Information (Fi...,/content/drive/MyDrive/Contact Information (Fi...,NaN,NaN,97092,0.936456
1,11/8/2023,Grant,KIA,PNJ2,https://drive.google.com/open?id=1GgqvsXuUFcNw...,241,https://drive.google.com/open?id=1s2j8tp5IDoqk...,245,trabajo,NaN,Work,4,4.76,NaN,/content/drive/MyDrive/Contact Information (Fi...,/content/drive/MyDrive/Contact Information (Fi...,NaN,NaN,gettyin,0.999954
